![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [4]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [5]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [6]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [7]:
cursor.description

(('name', None, None, None, None, None, None),)

In [8]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [9]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [10]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [11]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [9]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [12]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [13]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [14]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'alias', 'TEXT', 0, None, 0),
 (4, 'iata', 'TEXT', 0, None, 0),
 (5, 'icao', 'TEXT', 0, None, 0),
 (6, 'callsign', 'TEXT', 0, None, 0),
 (7, 'country', 'TEXT', 0, None, 0),
 (8, 'active', 'TEXT', 0, None, 0)]

In [15]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'airline', 'TEXT', 0, None, 0),
 (2, 'airline_id', 'TEXT', 0, None, 0),
 (3, 'source', 'TEXT', 0, None, 0),
 (4, 'source_id', 'TEXT', 0, None, 0),
 (5, 'dest', 'TEXT', 0, None, 0),
 (6, 'dest_id', 'TEXT', 0, None, 0),
 (7, 'codeshare', 'TEXT', 0, None, 0),
 (8, 'stops', 'TEXT', 0, None, 0),
 (9, 'equipment', 'TEXT', 0, None, 0)]

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [16]:
cursor.execute("""
SELECT *
FROM airlines
""")
cursor.fetchmany(5)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]

In [17]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [73]:
#Write a query that will join the latitude and longitude data from the airports table to the information on the routes table
cursor.execute("""Select latitude, longitude, airline, airline_id, source, source_id, dest, 
dest_id, codeshare, equipment, stops from airports cross join routes """).fetchmany(5)

[('-6.081689',
  '145.391881',
  '2B',
  '410',
  'AER',
  '2965',
  'KZN',
  '2990',
  None,
  'CR2',
  '0'),
 ('-6.081689',
  '145.391881',
  '2B',
  '410',
  'ASF',
  '2966',
  'KZN',
  '2990',
  None,
  'CR2',
  '0'),
 ('-6.081689',
  '145.391881',
  '2B',
  '410',
  'ASF',
  '2966',
  'MRV',
  '2962',
  None,
  'CR2',
  '0'),
 ('-6.081689',
  '145.391881',
  '2B',
  '410',
  'CEK',
  '2968',
  'KZN',
  '2990',
  None,
  'CR2',
  '0'),
 ('-6.081689',
  '145.391881',
  '2B',
  '410',
  'CEK',
  '2968',
  'OVB',
  '4078',
  None,
  'CR2',
  '0')]

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [20]:
# Which countries have the highest amount of active airports?
cursor.execute("select count(active), country from airlines where active = 'Y' group by country order by count(active) desc")
print(*cursor.fetchall(), sep='\n')

(141, 'United States')
(72, 'Russia')
(40, 'United Kingdom')
(37, 'Germany')
(34, 'Canada')
(26, 'Australia')
(25, 'China')
(24, 'Spain')
(23, 'Brazil')
(22, 'France')
(19, 'Japan')
(18, 'Italy')
(17, 'Indonesia')
(17, 'India')
(16, 'Turkey')
(16, 'Thailand')
(15, 'Sweden')
(14, 'Switzerland')
(13, 'Ukraine')
(13, 'Portugal')
(12, 'Mexico')
(12, 'Finland')
(12, 'Egypt')
(12, 'Austria')
(11, 'Peru')
(10, 'South Africa')
(10, 'Greece')
(10, 'Argentina')
(9, 'Netherlands')
(9, 'Iceland')
(9, 'Colombia')
(8, 'United Arab Emirates')
(8, 'Singapore')
(8, 'Romania')
(8, 'Poland')
(8, 'Philippines')
(7, 'Vietnam')
(7, 'Taiwan')
(7, 'Pakistan')
(7, 'Malaysia')
(7, 'Ireland')
(7, 'Iran')
(7, 'Ecuador')
(7, 'Denmark')
(6, 'South Korea')
(6, 'Norway')
(6, 'Nepal')
(6, 'Kazakhstan')
(6, 'Israel')
(6, 'Georgia')
(6, 'Chile')
(6, 'Burma')
(6, 'Belgium')
(5, 'Slovakia')
(5, 'Republic of Korea')
(5, 'Nigeria')
(5, 'Morocco')
(5, 'Lithuania')
(5, 'Dominican Republic')
(5, 'Czech Republic')
(5, 'Cambodia

In [21]:
# Which countries have the highest amount of inactive airports?
cursor.execute("select count(active), country from airlines where active = 'N' group by country order by count(active) desc")
print(*cursor.fetchall(), sep='\n')

(939, 'United States')
(427, 'Mexico')
(367, 'United Kingdom')
(284, 'Canada')
(158, 'Russia')
(142, 'Spain')
(97, 'France')
(94, 'Germany')
(81, 'South Africa')
(80, 'Nigeria')
(76, 'Ukraine')
(73, 'Kazakhstan')
(72, 'Italy')
(67, 'Australia')
(55, 'Sweden')
(47, 'Sudan')
(46, 'Switzerland')
(45, 'China')
(43, 'Netherlands')
(38, 'Austria')
(36, 'Egypt')
(35, 'Chile')
(35, 'Brazil')
(35, 'Angola')
(34, 'Colombia')
(32, 'Thailand')
(32, 'Portugal')
(32, 'Mauritania')
(31, 'Indonesia')
(29, 'Kyrgyzstan')
(27, 'Czech Republic')
(25, 'Japan')
(25, 'Dominican Republic')
(25, 'Denmark')
(25, 'Bulgaria')
(24, 'New Zealand')
(24, 'Iran')
(23, 'United Arab Emirates')
(23, 'Turkey')
(23, 'Sierra Leone')
(23, 'Norway')
(23, 'Libya')
(22, 'Venezuela')
(22, 'Pakistan')
(21, 'Uganda')
(21, 'Ireland')
(21, 'Belgium')
(20, 'Zambia')
(20, 'Kenya')
(19, 'Greece')
(19, 'Ecuador')
(19, 'Democratic Republic of the Congo')
(17, 'Moldova')
(17, 'Ghana')
(16, 'Israel')
(15, 'Peru')
(15, 'Ivory Coast')
(15, '

In [22]:
# What about airports by timezones?
cursor.execute(" select timezone, country from airports")
print(*cursor.fetchall(), sep='\n')

('Pacific/Port_Moresby', 'Papua New Guinea')
('Pacific/Port_Moresby', 'Papua New Guinea')
('Pacific/Port_Moresby', 'Papua New Guinea')
('Pacific/Port_Moresby', 'Papua New Guinea')
('Pacific/Port_Moresby', 'Papua New Guinea')
('Pacific/Port_Moresby', 'Papua New Guinea')
('America/Godthab', 'Greenland')
('America/Godthab', 'Greenland')
('America/Godthab', 'Greenland')
('America/Thule', 'Greenland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('Atlantic/Reykjavik', 'Iceland')
('America/Toronto', 'Canada')
('America/Winnipeg', 'Canada')
('America/Halifax', 'Canada')
('America/St_Johns', 'Canada')
('America/Vancouver', 'Canada')
('America/Edmonton', 'Canada')
('America/Toronto', 'Canada')
('America/Toronto', 'Canada')
('America/Winni

('America/Edmonton', 'Canada')
('Asia/Aden', 'Yemen')
('Asia/Aden', 'Yemen')
('Europe/Istanbul', 'Turkey')
('America/Cordoba', 'Argentina')
('Asia/Baghdad', 'Iraq')
('Australia/Brisbane', 'Australia')
('Europe/Athens', 'Greece')
('Asia/Tokyo', 'Japan')
('America/New_York', 'United States')
('Asia/Manila', 'Philippines')
('America/Godthab', 'Greenland')
('America/Godthab', 'Greenland')
('America/Godthab', 'Greenland')
('Europe/Madrid', 'Spain')
('Australia/Darwin', 'Australia')
('Asia/Bangkok', 'Thailand')
('Asia/Rangoon', 'Burma')
('Asia/Manila', 'Philippines')
('America/Nassau', 'Bahamas')
('America/Anchorage', 'United States')
('Europe/Malta', 'Malta')
('America/New_York', 'United States')
('America/New_York', 'United States')
('America/New_York', 'United States')
('America/Chicago', 'United States')
('Australia/Darwin', 'Australia')
('America/New_York', 'United States')
('America/New_York', 'United States')
('Asia/Kuala_Lumpur', 'Malaysia')
('America/New_York', 'United States')
('Am

('America/New_York', 'United States')
('America/New_York', 'United States')
('Europe/Berlin', 'Germany')
('America/Chicago', 'United States')
('America/New_York', 'United States')
('Europe/London', 'United Kingdom')
('Europe/Berlin', 'Germany')
('America/Denver', 'United States')
('Africa/Bamako', 'Mali')
('Australia/Sydney', 'Australia')
('America/New_York', 'United States')
('America/New_York', 'United States')
('Australia/Brisbane', 'Australia')
('Australia/Darwin', 'Australia')
('Australia/Brisbane', 'Australia')
('Australia/Brisbane', 'Australia')
('Pacific/Honolulu', 'United States')
('Asia/Kabul', 'Afghanistan')
('Asia/Chongqing', 'China')
('America/Los_Angeles', 'United States')
('Africa/Nairobi', 'Kenya')
('America/Guayaquil', 'Ecuador')
('Asia/Pyongyang', 'North Korea')
('Australia/Adelaide', 'Australia')
('Europe/Warsaw', 'Poland')
('America/Chicago', 'United States')
('America/New_York', 'United States')
('America/Toronto', 'Canada')
('America/Toronto', 'Canada')
('America/

In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [23]:
import pandas as pd

pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [24]:
pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airlines limit 5;", pd_con)
df

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [25]:
# Which countries have the highest amount of active airports?
conn = sqlite3.connect("data/flights.db")
sql = 'select count(a.active) as active, ar.country from airports ar join airlines a  on a.icao = ar.icao where a.active = "Y" group by ar.country order by count(a.active) desc'
pd.read_sql_query(sql, conn)

,active,country
0,45222,United States
1,7735,Germany
2,7259,Canada
3,6545,Russia
4,6545,China
...,...,...
123,119,Azerbaijan
124,119,Angola
125,119,American Samoa
126,119,Algeria


In [37]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [126]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [127]:
# How many tables are in the database?
cur.execute("SELECT count(name) FROM sqlite_master WHERE type='table'")
print(cur.fetchall())

# your code here

[(11,)]


In [128]:
# What's the primary key of each table?
cur.execute('''select p.name from PRAGMA_table_info('Artist') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key
# your code here

[('ArtistId',)]

In [129]:
cur.execute('''select p.name from PRAGMA_table_info('Album') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('AlbumId',)]

In [130]:
cur.execute('''select p.name from PRAGMA_table_info('Customer') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('CustomerId',)]

In [131]:
cur.execute('''select p.name from PRAGMA_table_info('Employee') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('EmployeeId',)]

In [132]:
cur.execute('''select p.name from PRAGMA_table_info('Genre') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('GenreId',)]

In [133]:
cur.execute('''select p.name from PRAGMA_table_info('Invoice') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('InvoiceId',)]

In [134]:
cur.execute('''select p.name from PRAGMA_table_info('Invoiceline') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('InvoiceLineId',)]

In [135]:
cur.execute('''select p.name from PRAGMA_table_info('MediaType') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('MediaTypeId',)]

Primary key - MediaTypeId

In [136]:
cur.execute('''select p.name from PRAGMA_table_info('Playlist') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('PlaylistId',)]

In [137]:
cur.execute('''select p.name from PRAGMA_table_info('PlaylistTrack') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('PlaylistId',)]

In [138]:
cur.execute('''select p.name from PRAGMA_table_info('Track') as p where p.pk = 1;''')
prim_key = cur.fetchall()
prim_key

[('TrackId',)]

In [139]:
# What foreign keys join the tables together?
cur.execute("PRAGMA foreign_key_list(Album)")
cur.fetchall()
# your code here
# hint: use "PRAGMA foreign_key_list()"


[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]

In [140]:
cur.execute("PRAGMA foreign_key_list(Artist)")
cur.fetchall()

[]

In [141]:
cur.execute("PRAGMA foreign_key_list(Customer)")
cur.fetchall()

[(0,
  0,
  'Employee',
  'SupportRepId',
  'EmployeeId',
  'NO ACTION',
  'NO ACTION',
  'NONE')]

In [142]:
cur.execute("PRAGMA foreign_key_list(Employee)")
cur.fetchall()

[(0,
  0,
  'Employee',
  'ReportsTo',
  'EmployeeId',
  'NO ACTION',
  'NO ACTION',
  'NONE')]

In [143]:
cur.execute("PRAGMA foreign_key_list(Genre)")
cur.fetchall()

[]

In [144]:
cur.execute("PRAGMA foreign_key_list(Invoice)")
cur.fetchall()

[(0,
  0,
  'Customer',
  'CustomerId',
  'CustomerId',
  'NO ACTION',
  'NO ACTION',
  'NONE')]

In [145]:
cur.execute("PRAGMA foreign_key_list(InvoiceLine)")
cur.fetchall()

[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'),
 (1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')]

In [146]:
cur.execute("PRAGMA foreign_key_list(MediaType)")
cur.fetchall()

[]

In [147]:
cur.execute("PRAGMA foreign_key_list(playlist)")
cur.fetchall()

[]

In [148]:
cur.execute("PRAGMA foreign_key_list(playlistrack)")
cur.fetchall()

[]

In [149]:
cur.execute("PRAGMA foreign_key_list(Track)")
cur.fetchall()

[(0,
  0,
  'MediaType',
  'MediaTypeId',
  'MediaTypeId',
  'NO ACTION',
  'NO ACTION',
  'NONE'),
 (1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE'),
 (2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')]

In [150]:
df = pd.read_sql_query("select * from Invoice limit 5;", conn)
df

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [151]:
# What are the max and min dates in the Invoice table?
sql = 'select max(InvoiceDate) as max_date, min(InvoiceDate) as min_date from Invoice order by InvoiceDate '
pd.read_sql_query(sql, conn)

,max_date,min_date
0,2013-12-22 00:00:00,2009-01-01 00:00:00


In [152]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cur.fetchall())

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [153]:
# What tables would you need to answer "what is your most popular track?"
df = pd.read_sql_query("select * from Track limit 5;", conn)
df


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [154]:
df = pd.read_sql_query("select * from playlistTrack where PlaylistId = 5 limit 20;", conn)
df


,PlaylistId,TrackId
0,5,3
1,5,4
2,5,5
3,5,23
4,5,24
5,5,25
6,5,26
7,5,27
8,5,28
9,5,29


# What values from each table?
playlistTrack, Track

In [159]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.
sql = "SELECT tr.name as Song_Name,COUNT(DISTINCT pt.playlistid) as count_of_playlist FROM playlisttrack as pt JOIN Track as tr ON pt.TrackId =tr.TrackId GROUP BY pt.trackid ORDER BY COUNT(pt.playlistid)  DESC"
pd.read_sql_query(sql, conn)

,Song_Name,count_of_playlist
0,Intoitus: Adorate Deum,5
1,"Miserere mei, Deus",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,"The Messiah: Behold, I Tell You a Mystery... T...",5
...,...,...
3498,October Song,2
3499,What Is It About Men,2
3500,Help Yourself,2
3501,Amy Amy Amy (Outro),2


In [164]:
# Advanced: get the artist who sang the song!
sql = '''
SELECT ar.name as Artist_Name ,tr.name as Song_Name,COUNT(DISTINCT pt.playlistid) as count_of_Playlist FROM playlistTrack as pt
JOIN Track as tr ON pt.TrackId =tr.TrackId
join album as al On al.albumid = tr.albumid
join artist as ar On ar.artistid = al.artistid GROUP BY pt.trackid ORDER BY COUNT(pt.playlistid)  DESC;
'''
pd.read_sql_query(sql, conn)

,Artist_Name,Song_Name,count_of_Playlist
0,Alberto Turco & Nova Schola Gregoriana,Intoitus: Adorate Deum,5
1,"Richard Marlow & The Choir of Trinity College,...","Miserere mei, Deus",5
2,Wilhelm Kempff,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,Yo-Yo Ma,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,Scholars Baroque Ensemble,"The Messiah: Behold, I Tell You a Mystery... T...",5
...,...,...,...
3498,Amy Winehouse,October Song,2
3499,Amy Winehouse,What Is It About Men,2
3500,Amy Winehouse,Help Yourself,2
3501,Amy Winehouse,Amy Amy Amy (Outro),2


#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_alexes
FROM student_courses
WHERE student_name = "Alex"
GROUP BY 1
HAVING COUNT(student_name) >= 2;
```

In [36]:
conn = sqlite3.connect('data/tutorial.db')
c = conn.cursor()

In [37]:
columns = [x[0] for x in c.execute('select * from responses').description]

In [38]:
columns

['submitted_ts',
 'name',
 'birthday',
 'hometown',
 'fav_food_1',
 'fav_food_2',
 'time_in_dc',
 'siblings_count']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [39]:
sql = '''
select * from responses
'''
pd.read_sql_query(sql, conn)

,submitted_ts,name,birthday,hometown,fav_food_1,fav_food_2,time_in_dc,siblings_count
0,12/11/2019 12:57:49,Stephen,12/8,"Arlington, TX",pizza,beef stroganoff,0.0,2
1,12/11/2019 13:04:35,Donna C,9/29,"Phoenix, AZ",shrimp,sushi,0.2,1
2,12/11/2019 13:05:08,Muoyo,6/22,"Brooklyn, NY",Nigerian,Thai,0.0,1
3,12/11/2019 13:06:13,Vyjayanthi,7/24,"Fairfax, VA",Pasta,Noodles,11.0,3
4,12/11/2019 13:21:20,Anesu Masube,3/28,"Kwekwe, Zimbabwe",Pork Ribs,Sadza,3.0,5
5,12/11/2019 13:38:17,Michael Pallante,8/5,"Voorhees, NJ",Pizza,Pasta,1.0,2
6,12/11/2019 13:43:36,Stuart Murphy,1/5,"London, England",Bread,Indian,10.0,1
7,12/11/2019 13:48:58,Darian Madere,8/7,"Baton Ruuge, LA",tater tots,sushi,1.0,3
8,12/11/2019 13:49:07,Alex,2/21,"Alexandria, VA",Ice cream,Carrot cake,3.0,1
9,12/11/2019 13:52:47,Justin Fleury,5/5,"Washington, DC",lasagna,brownies,1.0,1


In [40]:
sql = '''
select name from responses 
'''
pd.read_sql_query(sql, conn)

,name
0,Stephen
1,Donna C
2,Muoyo
3,Vyjayanthi
4,Anesu Masube
5,Michael Pallante
6,Stuart Murphy
7,Darian Madere
8,Alex
9,Justin Fleury


2. Which student has the most siblings?

In [41]:
sql = '''
select max(siblings_count), name from responses 
'''
pd.read_sql_query(sql, conn)

,max(siblings_count),name
0,12,Nick


3. How many students are only children?

In [42]:
sql = '''
select name from responses where siblings_count = 0
'''
pd.read_sql_query(sql, conn)

,name


4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [43]:
sql = '''
select name, time_in_dc from responses where hometown = "Washington, DC" order by time_in_dc limit 3;
'''
pd.read_sql_query(sql, conn)

,name,time_in_dc
0,Justin Fleury,1.0
1,Nate Lu,3.0


5. How many students are native New Yorkers?

In [44]:
sql = '''
select name, hometown from responses where hometown like "%NY"
'''
pd.read_sql_query(sql, conn)

,name,hometown
0,Muoyo,"Brooklyn, NY"
1,Nimu,"Nyack, NY"


6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [122]:
sql = '''
select f1.name, f2.name, f1.fav_food_1, f2.fav_food_1, f1.fav_food_2,  f2.fav_food_2 from responses f1, responses f2 
where f1.fav_food_1 = f2.fav_food_2 or f1.fav_food_1 = f2.fav_food_1 or f1.fav_food_2 = f2.fav_food_2
group by f1.name,f2.name
having f1.name <> f2.name
order by f1.fav_food_1 desc'''
pd.read_sql_query(sql, conn)

,name,name,fav_food_1,fav_food_1,fav_food_2,fav_food_2
0,Darian Madere,Donna C,tater tots,shrimp,sushi,sushi
1,Donna C,Darian Madere,shrimp,tater tots,sushi,sushi
2,Jill Carrie,Muoyo,Thai,Nigerian,Sushi,Thai
3,Nate Lu,Jill Carrie,Sushi,Thai,Grey,Sushi
4,Vyjayanthi,Michael Pallante,Pasta,Pizza,Noodles,Pasta


## More Questions

What are the favorite foods of this classroom?

In [62]:
sql = '''
select fav_food_1, fav_food_2 from responses
'''
pd.read_sql_query(sql, conn)

,fav_food_1,fav_food_2
0,pizza,beef stroganoff
1,shrimp,sushi
2,Nigerian,Thai
3,Pasta,Noodles
4,Pork Ribs,Sadza
5,Pizza,Pasta
6,Bread,Indian
7,tater tots,sushi
8,Ice cream,Carrot cake
9,lasagna,brownies


Which student was born closest to the cohort's graduation date (5/29/20)?

In [104]:
sql = '''
SELECT birthday
FROM responses r
WHERE r.birthday between 5/1 AND 6/1
ORDER BY r.birthday DESC
limit 1

'''
pd.read_sql_query(sql, conn)

,birthday
0,5/5
